In [1]:
import json
import requests
import time
import pandas as pd
import math
from pandas.io.json import json_normalize

In [2]:
# retrieve api key from file
api_key = json.loads(open('/tmp/config.json', 'r').read())["API_KEY"]

In [3]:
# retrieve all pages from article search on "javascript"
offset, max_offset = 0, None
q = 'javascript'

all_df = []
while max_offset == None or offset <= max_offset:
    res = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={q}&api-key={api_key}&page={offset}')
    parsed = json.loads(res.text)
    if max_offset == None:
        try:
            max_offset = math.ceil(float(parsed['response']['meta']['hits']) // 10)
        except:
            'no max offset'
            break
    try:
        docs = json_normalize(parsed['response']['docs'])
        print(parsed['response']['meta'], end='')
        print(' | OFFSET', offset);
        all_df.append(docs)
        offset += 1
        # sleep to avoid quota limit
        time.sleep(7)
    except KeyError:
        print('key error, could not find response, docs or meta in:', parsed)
        break
    

{'hits': 420, 'offset': 0, 'time': 160} | OFFSET 0
{'hits': 420, 'offset': 10, 'time': 136} | OFFSET 1
{'hits': 420, 'offset': 20, 'time': 147} | OFFSET 2
{'hits': 420, 'offset': 30, 'time': 127} | OFFSET 3
{'hits': 420, 'offset': 40, 'time': 180} | OFFSET 4
{'hits': 420, 'offset': 50, 'time': 132} | OFFSET 5
{'hits': 420, 'offset': 60, 'time': 159} | OFFSET 6
{'hits': 420, 'offset': 70, 'time': 187} | OFFSET 7
{'hits': 420, 'offset': 80, 'time': 206} | OFFSET 8
{'hits': 420, 'offset': 90, 'time': 134} | OFFSET 9
{'hits': 420, 'offset': 100, 'time': 199} | OFFSET 10
{'hits': 420, 'offset': 110, 'time': 147} | OFFSET 11
{'hits': 420, 'offset': 120, 'time': 157} | OFFSET 12
{'hits': 420, 'offset': 130, 'time': 166} | OFFSET 13
{'hits': 420, 'offset': 140, 'time': 166} | OFFSET 14
{'hits': 420, 'offset': 150, 'time': 104} | OFFSET 15
{'hits': 420, 'offset': 160, 'time': 188} | OFFSET 16
{'hits': 420, 'offset': 170, 'time': 111} | OFFSET 17
{'hits': 420, 'offset': 180, 'time': 191} | OFFSE

In [4]:
# put all dataframes together
df = pd.concat(all_df, sort=True)

In [5]:
df

,_id,abstract,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,...,print_section,pub_date,section_name,snippet,source,subsection_name,type_of_material,uri,web_url,word_count
0,nyt://article/c688f40d-357c-56e0-aace-94fda321...,“What’s so special about JavaScript?” I asked ...,None,By BRAD STENGER,"[{'firstname': 'Brad', 'middlename': None, 'la...",article,None,Open,JavaScript Meetup City,None,...,NaN,2012-04-04T15:34:49+0000,Blogs,“What’s so special about JavaScript?” I asked ...,The New York Times,NaN,News,nyt://article/c688f40d-357c-56e0-aace-94fda321...,https://open.blogs.nytimes.com/2012/04/04/java...,1428
1,nyt://article/b3994694-4032-52b1-82b5-a4e3c8d9...,Watch video from the August 2012 TimesOpen eve...,None,By Brad Stenger,"[{'firstname': 'Brad', 'middlename': None, 'la...",article,None,Open,"HTML5, Apps and JavaScript Video",None,...,NaN,2012-08-22T20:39:08+0000,Blogs,Watch video from the August 2012 TimesOpen eve...,The New York Times,NaN,News,nyt://article/b3994694-4032-52b1-82b5-a4e3c8d9...,https://open.blogs.nytimes.com/2012/08/22/html...,71
2,nyt://article/fccf7818-2db3-5d1d-b4e8-4692009c...,"When Google decided to create its Web browser,...",None,By George Gene Gustines,"[{'firstname': 'George', 'middlename': 'Gene',...",article,None,None,"Archie and Jughead? No, Javascript",None,...,C,2008-09-07T23:56:22+0000,Business Day,"When Google decided to create its Web browser,...",The New York Times,Media,News,nyt://article/fccf7818-2db3-5d1d-b4e8-4692009c...,https://www.nytimes.com/2008/09/08/business/me...,447
3,nyt://article/4020208e-53b1-5843-a093-0670531e...,The first TimesOpen event of 2012 was a big su...,None,By Eric Schorr,"[{'firstname': 'Eric', 'middlename': None, 'la...",article,None,Open,"HTML5, Apps and JavaScript Wrap-Up",None,...,NaN,2012-08-17T01:06:57+0000,Blogs,The first TimesOpen event of 2012 was a big su...,The New York Times,NaN,News,nyt://article/4020208e-53b1-5843-a093-0670531e...,https://open.blogs.nytimes.com/2012/08/16/html...,253
4,nyt://article/6b89bb4a-f726-5bdc-9adb-6d3a87ca...,Your strongest debugging tool is the one betwe...,None,By Andre Behrens,"[{'firstname': 'Andre', 'middlename': None, 'l...",article,None,Open,The Young Developer’s Guide to Debugging JavaS...,None,...,NaN,2013-08-27T17:37:43+0000,Blogs,Your strongest debugging tool is the one betwe...,The New York Times,NaN,News,nyt://article/6b89bb4a-f726-5bdc-9adb-6d3a87ca...,https://open.blogs.nytimes.com/2013/08/27/the-...,1635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,nyt://interactive/2d1aecb3-5448-548b-a3f1-432f...,After we wrote last week about a sweeping new ...,None,None,[],multimedia,None,None,"‘When I See Racial Disparities, I See Racism.’...",None,...,NaN,2018-03-27T09:00:02+0000,Reader Center,After we wrote last week about a sweeping new ...,The New York Times,NaN,Interactive Feature,nyt://interactive/2d1aecb3-5448-548b-a3f1-432f...,https://www.nytimes.com/interactive/2018/03/27...,0
6,nyt://article/5bd8d1ce-53f0-5417-9504-3d358b50...,Correction of article in October 7 Circuits se...,None,None,[],article,None,None,Corrections,None,...,A,1999-10-11T05:00:00+0000,New York,Correction of article in October 7 Circuits se...,The New York Times,NaN,Correction,nyt://article/5bd8d1ce-53f0-5417-9504-3d358b50...,https://www.nytimes.com/1999/10/11/nyregion/c-...,43
7,nyt://article/e648f314-3bfc-5e17-bc7a-2d90a55c...,Body cameras have been promoted as a solution ...,None,By McKenzie Funk,"[{'firstname': 'Mckenzie', 'middlename': None,...",article,None,Feature,Should We See Everything a Cop Sees?,None,...,MM,2016-10-18T09:00:07+0000,Magazine,Body cameras have been promoted as a solution ...,The New York Times,NaN,News,nyt://article/e648f314-3bfc-5e17-bc7a-2d90a55c...,https://www.nytimes.com/2016/10/23/magazine/po...,7758
8,nyt://article/08687039-e637-5421-b40a-956013d2...,David Davis letter on Oct 7 art

In [16]:
# although the json was flattened with normalized, arrays aren't converted to columns
df['keywords'][:10]

0                                                   []
1                                                   []
2    [{'name': 'organizations', 'value': 'Google In...
3                                                   []
4                                                   []
5    [{'name': 'subject', 'value': 'Insider Trading...
6                                                   []
7                                                   []
8    [{'name': 'subject', 'value': 'Tax Evasion', '...
9    [{'name': 'organizations', 'value': 'PRIVACY F...
Name: keywords, dtype: object

In [17]:
# view a single entry
# use iloc, since multiple rows will have index 2 due to concatenation
df['keywords'].iloc[2]

[{'name': 'organizations', 'value': 'Google Inc', 'rank': 1, 'major': 'N'},
 {'name': 'subject',
  'value': 'Comic Books and Strips',
  'rank': 2,
  'major': 'N'},
 {'name': 'subject',
  'value': 'Computers and the Internet',
  'rank': 3,
  'major': 'N'},
 {'name': 'subject', 'value': 'Web Browsers', 'rank': 4, 'major': 'N'}]

In [23]:
# flatten the keywords into a simple string joined by comma
df['keywords.normalized'] = df['keywords'].map(lambda arr: ','.join([d["value"] for d in arr]))

In [24]:
# show the keyword and normalized keywords columns (along with headline)
df[['headline.main', 'keywords', 'keywords.normalized']]

,headline.main,keywords,keywords.normalized
0,JavaScript Meetup City,[],
1,"HTML5, Apps and JavaScript Video",[],
2,"Archie and Jughead? No, Javascript","[{'name': 'organizations', 'value': 'Google In...","Google Inc,Comic Books and Strips,Computers an..."
3,"HTML5, Apps and JavaScript Wrap-Up",[],
4,The Young Developer’s Guide to Debugging JavaS...,[],
...,...,...,...
5,"‘When I See Racial Disparities, I See Racism.’...","[{'name': 'subject', 'value': 'Discrimination'...","Discrimination,Blacks,Income Inequality,Povert..."
6,Corrections,"[{'name': 'subject', 'value': 'COMPUTER SOFTWA...","COMPUTER SOFTWARE,Correction Stories,Computers..."
7,Should We See Everything a Cop Sees?,"[{'name': 'subject', 'value': 'Police Brutalit...","Police Brutality, Misconduct and Shootings,Fun..."
8,In Defense of Browser Traps,"[{'name': 'persons', 'value': 'Davis, David', ...","Davis, David,COMPUTER SOFTWARE,Computers and t..."
